In [1]:
import tensorflow as tf

**`Convolution Layer`**

In [2]:
# 创建 filter 的 weight & biaes
filter_weight = tf.get_variable(
    'weights',
    [5, 5, 3, 16],    # 前两个过滤器尺寸，第三个 - 当前层的深度，第四个 - 过滤器的深度
    initializer=tf.truncated_normal_initializer(stddev=0.1)
)

# 和卷积层的权重类似，当前层矩阵上不同位置的偏置项也是共享的
# 所以偏置个数与过滤器的个数相同
biases = tf.get_variable(
    'biases', [16], initializer=tf.constant_initializer(0.1)
)

In [3]:
if False:
    conv = tf.nn.conv2d(
        input,
        filter_weight,
        strides=[1,1,1,1],
        padding='SAME'
    )
    # input : 当前层的输入矩阵，4-dims, [m, ., ., .]
    # filter : 提供了卷积层的权重
    # strides : 不同维度上的步长
    #           第一个纬度 & 最后一个纬度一定要为 1
    #           因为卷积层的步长支队矩阵的长和宽有效
    # padding : 填充方法
    #           SAME - 全零填充
    #           VALID - 不填充

In [4]:
if False:
    bias = tf.nn.bias_add(conv, biases)
    atived_conv = tf.nn.relu(bias)
    # tf.nn.bias_add() 提供了一个方便的函数给每一个节点加上偏置项
    # 注意这里不能直接使用加法，因为矩阵上不同位置上的节点都需要加上同样的偏置项

**`Pooling Layer`**

In [5]:
if False:
    pool = tf.nn.max_pool(actived_conv, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')
    # ksize : 过滤器尺寸
    # strides : 步长信息
    # padding : 提供是否使用全 0 填充

**`LeNET-5`**

In [6]:
import tensorflow as tf
import numpy as np
import os

In [7]:
INPUT_NODE = 784
OUTPUT_NODE = 10

IMAGE_SIZE = 28
NUM_CHANNELS = 1
NUM_LABELS = 10

CONV1_DEEP = 32
CONV1_SIZE = 5
CONV2_DEEP = 64
CONV2_SIZE = 5
FC_SIZE = 512

In [8]:
# 定义卷积神经网络的前向传播过程
# train : 用于判断是 '训练过程'，'测试过程'
# 在训练中使用 dropout
def inference(input_tensor, USE, train, regularizer):
    # 通过使用不同的命名空间来隔离不同层的变量，这可以让每一层中的变量命名只需要考虑在当前层的作用，而不需要担心重命名的问题
    with tf.variable_scope('layer1-conv1', reuse=USE):
        conv1_weights = tf.get_variable(
            "weight",
            [CONV1_SIZE, CONV1_SIZE, NUM_CHANNELS, CONV1_DEEP],
            initializer=tf.truncated_normal_initializer(stddev=0.1)
        )
        conv1_biases = tf.get_variable(
            "bias",
            [CONV1_DEEP],
            initializer=tf.constant_initializer(0.0)
        )
        
        conv1 = tf.nn.conv2d(input_tensor, conv1_weights, strides=[1,1,1,1], padding='SAME')
        relu1 = tf.nn.relu(tf.nn.bias_add(conv1, conv1_biases))

    with tf.name_scope('layer2-pool1'):
        pool1 = tf.nn.max_pool(
            relu1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME'
        )
    
    with tf.variable_scope('layer3-conv2', reuse=USE):
        conv2_weights = tf.get_variable(
            "weight", [CONV2_SIZE, CONV2_SIZE, CONV1_DEEP, CONV2_DEEP],
            initializer=tf.truncated_normal_initializer(stddev=0.1)
        )
        conv2_biases = tf.get_variable(
            "bias", [CONV2_DEEP], initializer=tf.constant_initializer(0.0)
        )
        conv2 = tf.nn.conv2d(pool1, conv2_weights, strides=[1,1,1,1], padding='SAME')
        relu2 = tf.nn.relu(tf.nn.bias_add(conv2, conv2_biases))
    
    with tf.name_scope("layer4-pool2"):
        pool2 = tf.nn.max_pool(
            relu2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME'
        )

    # 将第四层池化层的输出转化为第五层全连接层的输入格式
    # pool2.get_shape() 可以得到第四层输出矩阵的纬度，而不需要手工计算
    # 因为每一层神经网络的输入输出都为一个 batch 的矩阵，
    # 所以这里得到的纬度也包含了一个 batch 中数据的个数
    pool_shape = pool2.get_shape().as_list()
    # pool_shape[0] 是一个 batch 中的数据个数
    nodes = pool_shape[1] * pool_shape[2] * pool_shape[3]
    
    # 通过 tf.reshape 函数将第四层的输出变成一个 batch 的向量
    reshaped = tf.reshape(pool2, [pool_shape[0], nodes])
    
    # 声明第五层全连接层的变量，并实现向前传播过程
    # dropout 一般只会在全连接层而不是卷积层或是池化层使用
    with tf.variable_scope('layer5-fc1', reuse=USE):
        fc1_weights = tf.get_variable(
            "weight", [nodes, FC_SIZE],
            initializer=tf.truncated_normal_initializer(stddev=0.1)
        )
        # 只有全连接层的权重需要加入正则化
        if regularizer != None:
            tf.add_to_collection('losses', regularizer(fc1_weights))
        fc1_biases = tf.get_variable(
            "bias", [FC_SIZE], initializer=tf.constant_initializer(0.1)
        )
        
        fc1 = tf.nn.relu(tf.matmul(reshaped, fc1_weights) + fc1_biases)
        if train:
            fc1 = tf.nn.dropout(fc1, 0.5)
    
    with tf.variable_scope('layer6-fc2', reuse=USE):
        fc2_weights = tf.get_variable(
            "weight", [FC_SIZE, NUM_LABELS],
            initializer=tf.truncated_normal_initializer(stddev=0.1)
        )
        if regularizer != None:
            tf.add_to_collection('losses', regularizer(fc2_weights))
        fc2_biases = tf.get_variable(
            "bias", [NUM_LABELS],
            initializer=tf.constant_initializer(0.1)
        )
        
        logit = tf.matmul(fc1, fc2_weights) + fc2_biases
        
    return logit

**`train`**

In [9]:
BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.01
LEARNING_RATE_DECAY = 0.99
TRAINING_STEPS = 10000
MOVING_AVERAGE_DECAY = 0.9
LAMADA = 0.0001

In [10]:
MODEL_SAVE_PATH = 'model/'
MODEL_NAME = 'LeNet'

In [11]:
def train(mnist):
    x  = tf.placeholder(tf.float32, shape=(BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS), name='x-input')
    y_ = tf.placeholder(tf.float32, shape=(None, OUTPUT_NODE), name='y-input')
    regularizer = tf.contrib.layers.l2_regularizer(LAMADA)
    y = inference(x, False, False, regularizer)
    
    global_step = tf.Variable(0, trainable=False)
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variable_averages_op = variable_averages.apply(tf.trainable_variables())
    average_y = inference(x, True, False, None)
    
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_,1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    loss = tf.add_n(tf.get_collection('losses')) + cross_entropy_mean 

    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE,
                                               global_step,
                                               mnist.train.num_examples / BATCH_SIZE,
                                               LEARNING_RATE_DECAY,
                                               staircase=True)
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    with tf.control_dependencies([train_step, variable_averages_op]):
        train_op = tf.no_op(name='train')
    
    correct_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    with tf.Session() as sess:
        tf.global_variables_initializer().run()
      
        for i in range(TRAINING_STEPS):
            xv, yv = mnist.validation.next_batch(BATCH_SIZE)
            reshape_xv = np.reshape(xv, (BATCH_SIZE,
                                         IMAGE_SIZE,
                                         IMAGE_SIZE,
                                         NUM_CHANNELS))
            validate_feed = {x : reshape_xv, y_ : yv}
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            reshape_xs = np.reshape(xs, (BATCH_SIZE,
                                         IMAGE_SIZE,
                                         IMAGE_SIZE,
                                         NUM_CHANNELS))
            _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x : reshape_xs, y_ : ys})
            if i % 100 == 0:
                print("After %d training step(s), loss on training batch is %g." % (step, loss_value))
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                print("After %d training step(s),validation accuracy using average model is %g "%(step,validate_acc))

In [12]:
from tensorflow.examples.tutorials.mnist import input_data

In [13]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [14]:
train(mnist)

After 1 training step(s), loss on training batch is 3.28315.
After 1 training step(s),validation accuracy using average model is 0.16 
After 101 training step(s), loss on training batch is 1.02252.
After 101 training step(s),validation accuracy using average model is 0.84 
After 201 training step(s), loss on training batch is 0.889773.
After 201 training step(s),validation accuracy using average model is 0.89 
After 301 training step(s), loss on training batch is 0.740521.
After 301 training step(s),validation accuracy using average model is 0.93 
After 401 training step(s), loss on training batch is 1.01881.
After 401 training step(s),validation accuracy using average model is 0.93 
After 501 training step(s), loss on training batch is 0.759579.
After 501 training step(s),validation accuracy using average model is 0.93 
After 601 training step(s), loss on training batch is 0.769022.
After 601 training step(s),validation accuracy using average model is 0.94 
After 701 training step(s),

KeyboardInterrupt: 

In [ ]:
# 为什么一定需要带上 reuse, 不太懂